In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn.functional as functional
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# Define the image transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define device to be used (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained models
resnet18 = models.resnet18(pretrained=True).to(device)
vgg19 = models.vgg19(pretrained=True).to(device)
inception = models.inception_v3(pretrained=True).to(device)

# Set all models to eval mode (not training mode)
resnet18.eval();
vgg19.eval();
inception.eval();

In [3]:
phase = 'train'
train_path = 'code_test_data/'+ phase

# Load the custom dataset
dataset = ImageFolder(train_path, transform=transform)

# Get a list of unique class labels
classes = list(set(dataset.classes))

Use torch.no_grad(): If you're not updating the parameters of your model during evaluation, you can use the torch.no_grad() context manager to avoid keeping track of the gradients. This can reduce memory usage significantly.

Creation of dictionary 

In [4]:
# Create a temp dictionary to store feature vectors for each class
temp_dict = {}
for cls in classes:
    temp_dict[cls] = []

# Initialize new dictionary with null values
features_vgg = {key: None for key in temp_dict}
features_resnet = {key: None for key in temp_dict}
features_inception = {key: None for key in temp_dict}

# Iterate over the dataset and extract feature vectors for each class
loader = DataLoader(dataset, batch_size=1, shuffle=True)
for images, labels in loader:

    # Send the labels to the GPU
    labels = labels.to(device)
    # Send the images to the GPU
    images = images.to(device)
    
    cls = dataset.classes[labels[0]]
    
    with torch.no_grad():
        # for resnet18
        features_tensor = resnet18(images)
        features_vector_resnet = functional.normalize(features_tensor, p=2, dim=-1)

        # for vgg19
        features_tensor = vgg19(images)
        features_vector_vgg = functional.normalize(features_tensor, p=2, dim=-1)

        # for inception
        features_tensor = inception(images)
        features_vector_inception = functional.normalize(features_tensor, p=2, dim=-1)

        # # Print the number of rows
        # print(num_rows)
        if features_vgg[cls] is None:
            # print('features[cls] exists but has an empty/null value')
            features_vgg[cls] = features_vector_vgg
            features_resnet[cls] = features_vector_resnet
            features_inception[cls] = features_vector_inception
        else:
            # print('features[cls] exists and contains a non-empty/non-null value')
            features_vgg[cls] = torch.cat((features_vgg[cls], features_vector_vgg), dim=0)  
            features_resnet[cls] = torch.cat((features_resnet[cls], features_vector_resnet), dim=0) 
            features_inception[cls] = torch.cat((features_inception[cls], features_vector_inception), dim=0)   


In [5]:
# features_vgg[cls]

In [7]:
# Compute the average feature vector for each class
for cls in classes:
    features_vgg[cls] = torch.mean(features_vgg[cls], dim=0).unsqueeze(0)
    features_resnet[cls] = torch.mean(features_resnet[cls], dim=0).unsqueeze(0)
    features_inception[cls] = torch.mean(features_inception[cls], dim=0).unsqueeze(0)


Test module

In [ ]:
phase = 'test'
test_path = 'code_test_data/'+ phase

# Load the custom dataset
test_dataset = ImageFolder(test_path, transform=transform)

# Get a list of unique class labels
classes = list(set(test_dataset.classes))

In [10]:
# Create a dictionary to store one image per class
img_dict = {}

# Iterate over the dataset and extract feature vectors for each class
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [11]:
for img, label in test_loader:

  # Send the labels to the GPU
  label = label.to(device)
  # Send the images to the GPU
  img = img.to(device)  
  
  if label not in img_dict:
    img_dict[label] = img
  # Break the loop if all classes are found
  if len(img_dict) == len(classes): # num_classes is the number of classes in your dataset
    break

In [ ]:
# Choose one random image per class and compare its feature vector with others
correct = 0
total = len(classes)

temp_dict = {}
for cls in classes:
    temp_dict[cls] = []

# Initialize new dictionary with null values
resnet_similarities = {key: None for key in temp_dict}
vgg_similarities = {key: None for key in temp_dict}
inception_similarities = {key: None for key in temp_dict}

for cls in classes:
    temp_dict[cls] = []

for label,image in img_dict.items():

    # Send the labels to the GPU
    label = label.to(device)
    # Send the images to the GPU
    image = image.to(device)
    
    present_probe_cls = label
    
    with torch.no_grad():
        # for resnet18
        features_tensor = resnet18(image)
        features_vector_resnet = functional.normalize(features_tensor, p=2, dim=-1)
        # for vgg19
        features_tensor = vgg19(image)
        features_vector_vgg = functional.normalize(features_tensor, p=2, dim=-1)
        # for inception
        features_tensor = inception(image)
        features_vector_inception = functional.normalize(features_tensor, p=2, dim=-1)   
        
        # Compute the cosine similarity between the feature vector and the average feature vector for each class
        # Create a CosineSimilarity object with dim=0
        cos_sim = torch.nn.CosineSimilarity(dim=0)
        for cls in classes:
            temp = cos_sim(features_vector_resnet.unsqueeze(0), features_resnet[cls].unsqueeze(0))
            resnet_similarities[cls] = torch.mean(temp).unsqueeze(0)

            temp = cos_sim(features_vector_vgg.unsqueeze(0), features_vgg[cls].unsqueeze(0))
            vgg_similarities[cls] = torch.mean(temp).unsqueeze(0)

            temp = cos_sim(features_vector_inception.unsqueeze(0), features_inception[cls].unsqueeze(0))
            inception_similarities[cls] = torch.mean(temp).unsqueeze(0)

        # Compute the avg similarity for each class
        avg_similarity = {}
        for cls in classes:
            avg_similarity[cls] = (resnet_similarities[cls] + vgg_similarities[cls] + inception_similarities[cls])/3
        # Make a prediction based on the class with the highest cosine similarity
        max_key = max(avg_similarity, key=avg_similarity.get)
        # here max_key is the predicted class
        if present_probe_cls == max_key:
            correct += 1
        # total += 1
    # print("value of correct is", correct)
# Print the accuracy
print('Accuracy: {:.2f}%'.format(correct / total * 100))